In [1]:
import pandas as pd
import sqlite3
import altair as alt
import geopandas as gpd

# Regional Analysis

## Total Production

In [3]:
db_name = 'field_crops.db'
table = 'midwest_key_field_crops_cleaned'

query = f"""
Select 
    commodity_desc,
    year, 
    sum(value) as total_prod
from {table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year >= 1975
group by 
    commodity_desc, year
"""

conn = sqlite3.connect(db_name) 
result = pd.read_sql(query, conn)


# colors for the commodities
color_scale = alt.Scale(domain=['CORN', 'SOYBEANS', 'WHEAT'],
                        range=['#FFB14E', '#FA8775', '#B5E384'])


chart = alt.Chart(result).mark_area().encode(
    x=alt.X('year:O', axis=alt.Axis(title='Year')),
    y=alt.Y('total_prod:Q', axis=alt.Axis(title='Total Production (in BUs)')),
    color=alt.Color('commodity_desc:N', scale=color_scale, legend=alt.Legend(title="Commodity"))
).properties(
    title=alt.TitleParams(
        text='Total Crop Production Over Time',
        subtitle='Corn, Soybeans, and Wheat (in BU)',
        anchor='middle'
    ),
    width=600,
    height=400
)

chart


alt.Chart(...)

## Total Area Planted


In [4]:
db_name = 'field_crops.db'
table = 'midwest_area_planted_cleaned'

query = f"""
Select 
    commodity_desc,
    year, 
    sum(value) as total_area_planted,
    count(*) as num_counties
from {table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year >= 1975
group by 
    commodity_desc, year
"""

conn = sqlite3.connect(db_name) 
result = pd.read_sql(query, conn)

chart = alt.Chart(result).mark_area().encode(
    x=alt.X('year:O', axis=alt.Axis(title='Year')),
    y=alt.Y('total_area_planted:Q', axis=alt.Axis(title='Total Area Planted (in Acres)')),
    color=alt.Color('commodity_desc:N', scale=color_scale, legend=alt.Legend(title="Commodity"))   
    ).properties(
        title=alt.TitleParams(
            text='Total Area Planted Over Time in Acres',
            subtitle='Corn, Soybeans, and Wheat',
            anchor='middle'
        ),
        width=600,
        height=400
    )

chart

alt.Chart(...)

# State Analysis


## Production

### Aggregated by State

In [7]:
db_name = 'field_crops.db'
table = 'midwest_key_field_crops_cleaned'

query = f"""
Select 
    state_alpha,
    year, 
    sum(value) as total_prod
from {table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year >= 1975
group by 
    state_alpha, year
"""

conn = sqlite3.connect(db_name) 
result = pd.read_sql(query, conn)

chart = alt.Chart(result).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(title='Year')),
    y=alt.Y('total_prod:Q', axis=alt.Axis(title='Total Production (in BU)')),
    color=alt.Color('state_alpha:N', legend=alt.Legend(title="State"))   
    ).properties(
        title='Total Production over Time',
        width=600,
        height=400
    )

chart

alt.Chart(...)

## Area Planted

### Aggregated by State

In [9]:
db_name = 'field_crops.db'
table = 'midwest_area_planted_cleaned'

query = f"""
Select 
    state_alpha,
    year, 
    sum(value) as total_prod
from {table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year >= 1975
group by 
    state_alpha, year
"""

conn = sqlite3.connect(db_name) 
result = pd.read_sql(query, conn)

# Create the Altair line chart
chart = alt.Chart(result).mark_line().encode(
    x='year:O',  # Treat 'year' as an ordinal value (categorical)
    y='total_prod:Q',
    color='state_alpha:N'        
    ).properties(
        title='Total Acres Planted by State Over Time',
        width=600,
        height=400
    )
chart

alt.Chart(...)

### Land Usage Change by state

In [8]:
db_name = 'field_crops.db'
table = 'midwest_area_planted_cleaned'

query = f"""
Select 
    avg(value) AS Value_20,
    commodity_desc,
    state_alpha
from {table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_alpha, commodity_desc
"""
conn = sqlite3.connect(db_name) 
avg_area_2015_2020 = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS Value_70,
    commodity_desc,
    state_alpha
from {table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_alpha, commodity_desc
"""
avg_area_1975_1980 = pd.read_sql(query, conn)

prod_change = pd.merge(avg_area_2015_2020, avg_area_1975_1980, on=["commodity_desc", "state_alpha"])
prod_change["abs_change_in_area_planted"] = (prod_change['Value_20'] -  prod_change['Value_70']) 
prod_change["perc_change_in_area_planted"] = ((prod_change['Value_20'] -  prod_change['Value_70']) / prod_change['Value_70'])*100

chart = alt.Chart(prod_change).mark_bar().encode(
    x=alt.X('state_alpha:O', title='State'),
    y=alt.Y('abs_change_in_area_planted:Q', title='Change in Area Planted (in Acres)'),
    color=alt.Color('commodity_desc:N', scale=color_scale, legend=alt.Legend(title="Commodity"))  , 
    xOffset='commodity_desc:N'
    ).properties(
        title=alt.TitleParams(
            text='Absolute Change in Area Planted Over Time in Acres',
            subtitle='Land used for Corn, Soybeans, and Wheat',
            anchor='middle'
        ),
        width=600,
        height=400
    )

chart

alt.Chart(...)

# County Level


In [10]:
from vega_datasets import data

# Load U.S. states and counties
states = data.us_10m.url  # URL for U.S. states
counties = data.us_10m.url  # URL for U.S. counties
print(states)
print(counties)
states_gdf = gpd.read_file(states)
counties_gdf = gpd.read_file(counties)

https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json
https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json


/Users/jacobtrout/Library/Caches/pypoetry/virtualenvs/ag-data-viz-5d2VArwu-py3.12/lib/python3.12/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'us-10m.json': 'counties' (default), 'states', 'TopoJSON'. Specify layer parameter to avoid this warning.
  result = read_func(
/Users/jacobtrout/Library/Caches/pypoetry/virtualenvs/ag-data-viz-5d2VArwu-py3.12/lib/python3.12/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'us-10m.json': 'counties' (default), 'states', 'TopoJSON'. Specify layer parameter to avoid this warning.
  result = read_func(


## Production Change by County

In [12]:
db_name = 'field_crops.db'
table = 'midwest_key_field_crops_cleaned'

query = f"""
Select 
    avg(value) AS avg_value_present,
    commodity_desc,
    state_alpha, 
    state_ansi|| county_ansi as id
from {table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_ansi|| county_ansi, commodity_desc
"""
conn = sqlite3.connect(db_name) 
avg_area_present = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_value_past,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {table} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_ansi|| county_ansi, commodity_desc
"""
avg_area_past = pd.read_sql(query, conn)

prod_change = pd.merge(avg_area_present, avg_area_past, on=["commodity_desc", "id"])
prod_change["abs_change_prod"] = (prod_change['avg_value_present'] -  prod_change['avg_value_past']) 
prod_change["perc_change_prod"] = ((prod_change['avg_value_present'] -  prod_change['avg_value_past']) / prod_change['avg_value_past'])*100
prod_change

query = f"""
Select 
    distinct
    state_ansi
from {table} 
"""
conn = sqlite3.connect(db_name) 
check = pd.read_sql(query, conn)

state_ansi_list = check.iloc[:,0].to_list()
midwest_counties_gdf = counties_gdf[counties_gdf['id'].str[:2].isin(state_ansi_list)]
midwest_counties_gdf = midwest_counties_gdf[
    counties_gdf['id'].str[:2].isin(state_ansi_list) & 
    (counties_gdf['id'].str.len() == 5)  
]


merged = gpd.GeoDataFrame(pd.merge(prod_change, midwest_counties_gdf, on='id', how='left'))
merged.set_geometry('geometry', inplace=True)

crop_list = [ 'CORN', 'SOYBEANS', 'WHEAT']

for crop in crop_list:
    crop_df = merged[merged['commodity_desc']== crop]

    # background map
    county_map_background = alt.Chart(midwest_counties_gdf).mark_geoshape(
        fill='lightgray',  
        stroke='black',   
        strokeWidth=0.5    
    ).properties(
        width=800,
        height=500
    ).project('albersUsa')  

    # filled map
    county_map_filled = alt.Chart(crop_df).mark_geoshape(
        stroke='black',   
        strokeWidth=0.5 
    ).encode(
        color=alt.Color(
            'abs_change_prod:Q',
            scale=alt.Scale(
                domain=[-max(abs(crop_df["abs_change_prod"])), 0, max(abs(crop_df["abs_change_prod"]))],  # Centering around 0
                range=['red', 'white', 'blue']  # Blue for positive, white for zero, red for neg
            ),
            legend=alt.Legend(title='Change in Total Production')
        ),
        tooltip=['id:N', 'abs_change_prod:Q'] 
    ).properties(
        title=f'Map of Absolute Change Production for {crop}'
    ).project('albersUsa')

    # layer the filled map on top of the gray background
    layered_map = county_map_background + county_map_filled

    layered_map.show()


/Users/jacobtrout/Library/Caches/pypoetry/virtualenvs/ag-data-viz-5d2VArwu-py3.12/lib/python3.12/site-packages/geopandas/geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

## Yield Heat Map 

In [15]:
db_name = 'field_crops.db'
table_prod = 'midwest_key_field_crops_cleaned'
table_area = 'midwest_area_planted_cleaned'

query = f"""
Select 
    avg(value) AS avg_prod_present,
    commodity_desc,
    state_alpha, 
    state_ansi|| county_ansi as id
from {table_prod} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_ansi|| county_ansi, commodity_desc
"""

conn = sqlite3.connect(db_name) 
avg_prod_present = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_prod_past,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {table_prod} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_ansi|| county_ansi, commodity_desc
"""
conn = sqlite3.connect(db_name) 
avg_prod_past = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_area_present,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {table_area} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 2018 and 2023
group by state_ansi|| county_ansi , commodity_desc
"""
conn = sqlite3.connect(db_name) 
avg_area_present = pd.read_sql(query, conn)

query = f"""
Select 
    avg(value) AS avg_area_past,
    commodity_desc,
    state_alpha,
    state_ansi|| county_ansi as id
from {table_area} 
where short_desc != 'CORN, SILAGE - PRODUCTION, MEASURED IN TONS'
and asd_code != 99
and county_ansi != ""
and year between 1975 and 1980
group by state_ansi|| county_ansi , commodity_desc
"""
avg_area_past = pd.read_sql(query, conn)

avg_yield_past = pd.merge(avg_prod_past, avg_area_past, on=["commodity_desc", "id", "state_alpha"])
avg_yield_past["yield_past"] = (avg_yield_past['avg_prod_past'] / avg_yield_past['avg_area_past']) 

avg_yield_present = pd.merge(avg_prod_present, avg_area_present, on=["commodity_desc", "id", "state_alpha"])
avg_yield_present["yield_present"] = (avg_yield_present['avg_prod_present'] / avg_yield_present['avg_area_present']) 

yield_change = pd.merge(avg_yield_past, avg_yield_present, on=["commodity_desc", "id", "state_alpha"])

yield_change["abs_change_yield"] = (yield_change['yield_present'] -  yield_change['yield_past']) 
yield_change["perc_change_yield"] = ((yield_change['yield_present'] -  yield_change['yield_past']) / yield_change['yield_past'])*100

query = f"""
Select 
    distinct
    state_ansi
from {table} 
"""
conn = sqlite3.connect(db_name) 
check = pd.read_sql(query, conn)

state_ansi_list = check.iloc[:,0].to_list()
midwest_counties_gdf = counties_gdf[counties_gdf['id'].str[:2].isin(state_ansi_list)]
midwest_counties_gdf = midwest_counties_gdf[
    counties_gdf['id'].str[:2].isin(state_ansi_list) &  
    (counties_gdf['id'].str.len() == 5) 
]


midwest_counties_gdf['centroid'] = midwest_counties_gdf['geometry'].centroid
midwest_counties_gdf['longitude'] = midwest_counties_gdf['centroid'].x
midwest_counties_gdf['latitude'] = midwest_counties_gdf['centroid'].y


merged = gpd.GeoDataFrame(pd.merge(yield_change, midwest_counties_gdf, on='id', how='inner'))
merged.set_geometry('geometry', inplace=True)


heatmap_df = merged[['longitude', 'latitude', 'abs_change_yield']]
heatmap = (
    alt.Chart(heatmap_df)
    .mark_rect()
    .encode(
        x=alt.X('longitude:Q', bin=alt.Bin(maxbins=30), title='Longitude'),
        y=alt.Y('latitude:Q', bin=alt.Bin(maxbins=30), title='Latitude'),
        color=alt.Color('mean(abs_change_yield):Q', scale=alt.Scale(scheme='viridis'), title='Value'),
        tooltip=['longitude:Q', 'latitude:Q', 'mean(abs_change_yield):Q']
    )
    .properties(
        width=600,
        height=400,
        title='Heat Map of average change in yeild by Longitude and Latitude'
    )
)

heatmap.display()

/Users/jacobtrout/Library/Caches/pypoetry/virtualenvs/ag-data-viz-5d2VArwu-py3.12/lib/python3.12/site-packages/geopandas/geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


alt.Chart(...)